In [1]:
import random 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import math
import os

In [7]:
class error():
    def __init__(self,p):
        self.flag_exist = 0
        self.flag1 = 0 # 使われていないなら0、使用後は1
        self.flag2 = 0 # 実行中のみ1
        self.prob = p # 確率

In [ ]:
def x_error(p):
    error = error(p)
    if error.flag1 == 0:
        error.flag2 = 1
        error.flag2 = 0
        error.flag2 = 1

In [28]:
#エラーの定義 qubitは三次元配列で1つ目のインデックスでXかZか、2,3個目のインデックスで位置を指定
def x_error(qubit,i,j):
    qubit[0][i][j] = (qubit[0][i][j]+1)%2
def y_error(qubit,i,j):
    qubit[0][i][j] = (qubit[0][i][j]+1)%2    
    qubit[1][i][j] = (qubit[1][i][j]+1)%2
def z_error(qubit,i,j):
    qubit[1][i][j] =  (qubit[1][i][j]+1)%2 

def single_biased(qubit,i,j,p): # etaはバイアス
    p_x = p 
    p_z = p 
    prob = random.random() 
    if prob < p_z: #Z error
        z_error(qubit,i,j)
    elif prob < p_z+p_x: # X error
        x_error(qubit,i,j)
    elif prob < p_z+2*p_x: # Y error
        y_error(qubit,i,j)

def reversed_single_biased(qubit,i,j,p): # 表面符号と反復符号で逆になっているエラー
    p_x = p 
    p_z = p 
    prob = random.random() 
    if prob < p_z: #Z error
        x_error(qubit,i,j)
    elif prob < p_z+p_x: # X error
        z_error(qubit,i,j)
    elif prob < p_z+2*p_x: # Y error
        y_error(qubit,i,j)

def bitflip_error(qubit,i,j,p): # etaはバイアス
    p_x = p 
    p_z = p 
    prob = random.random() 
    if prob < 2*p_x: #Z error
        x_error(qubit,i,j)

def p_x_error(qubit,i,j,p):
    prob = random.random() 
    if prob < p:
        x_error(qubit,i,j)

def p_z_error(qubit,i,j,p):
    prob = random.random() 
    if prob < p:
        z_error(qubit,i,j)

# i番目にHadamard gateを作用させる
def H(qubit,i,j):
    qubit[0][i][j] = (qubit[0][i][j] + qubit[1][i][j]) % 2
    qubit[1][i][j] = (qubit[1][i][j] + qubit[0][i][j]) % 2
    qubit[0][i][j] = (qubit[0][i][j] + qubit[1][i][j]) % 2

#i番目がcontrolビット j 番目がtargetビットのCNOTgate
def CNOT(qubit_c,i,j,qubit_t,k,l):     #c, tには二次元[][]を代入する
    qubit_t[0][k][l] = (qubit_t[0][k][l] + qubit_c[0][i][j])%2 #コントロール側のXエラーはターゲットに
    qubit_c[1][i][j] = (qubit_c[1][i][j] + qubit_t[1][k][l])%2 #ターゲット側のZエラーはコントロールに

def p_x_error(error_count):
        error_count += 1
def p_z_error(error_count):
        error_count += 1

def generate_weight(code_distance,p_list,round_sur):

    qubits_d = np.zeros((2,code_distance,code_distance)) #データ量子ビットの格納
    qubits_d_X = np.zeros((round_sur+2,code_distance,code_distance)) #全体でのXエラーの履歴
    qubits_d_Z = np.zeros((round_sur+2,code_distance,code_distance)) #全体でのXエラーの履歴
    qubits_m_in = np.zeros((2,code_distance-1,code_distance-1)) #測定量子ビット(中)の数
    qubits_m_out_X = np.zeros((2,2,int((code_distance-1)/2))) #測定量子ビット(外)の数
    qubits_m_out_Z = np.zeros((2,2,int((code_distance-1)/2))) #測定量子ビット(外)の数

    syndrome_in_X = np.zeros((round_sur+2, code_distance-1, code_distance-1)) #シンドローム測定の回数+最初の状態のシンドローム+最後の測定から計算したシンドローム
    syndrome_in_Z = np.zeros((3, code_distance-1, code_distance-1)) #最初、最後のシンドローム測定、最後のデータの測定のみ
    syndrome_out_X = np.zeros((round_sur+2,2,int((code_distance-1)/2)))
    syndrome_out_Z = np.zeros((3,2,int((code_distance-1)/2)))

    #############  ループ部分  ##################
    error_count = 0
    for num in range(round_sur):
        
        ### 反復符号でのエラー
        for i in range(code_distance):
            for j in range(code_distance):
                p_x_error(error_count)
                p_z_error(error_count)
        
        ### アンシラにエラー(Xシンドロームのみ)
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j) % 2 == 0:
                    p_x_error(error_count)
                    p_z_error(error_count)
        for i in range(int((code_distance-1)/2)):
            p_x_error(error_count)
            p_z_error(error_count)
            p_x_error(error_count)
            p_z_error(error_count)

        ### Xシンドローム
        # 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j)%2 == 0: 
                    CNOT(qubits_m_in,i,j,qubits_d,i,j)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    CNOT(qubits_m_in,i,j,qubits_d,i,j+1)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    CNOT(qubits_m_in,i,j,qubits_d,i+1,j)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    CNOT(qubits_m_in,i,j,qubits_d,i+1,j+1)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    p_x_error(error_count)
                    p_z_error(error_count)
        # 外側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if j == 0:
                    if i % 2 == 1:
                        CNOT(qubits_m_out_X,0,int((i-1)/2),qubits_d,i,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        CNOT(qubits_m_out_X,0,int((i-1)/2),qubits_d,i+1,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        p_z_error(error_count)
                if j == code_distance-2:
                    if i % 2 == 0:
                        CNOT(qubits_m_out_X,1,int(i/2),qubits_d,i,code_distance-1)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        CNOT(qubits_m_out_X,1,int(i/2),qubits_d,i+1,code_distance-1)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)  
                        p_z_error(error_count)   

        #Zシンドローム(最後だけ)
        if num == round_sur - 1:
            # 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):    
                    if (i+j)%2 == 1: 
                        CNOT(qubits_d,i,j,qubits_m_in,i,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        #p_z_error(error_count)
                        CNOT(qubits_d,i,j+1,qubits_m_in,i,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        #p_z_error(error_count)
                        CNOT(qubits_d,i+1,j,qubits_m_in,i,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        #p_z_error(error_count)
                        CNOT(qubits_d,i+1,j+1,qubits_m_in,i,j)
                        p_x_error(error_count)
                        p_z_error(error_count)
                        p_x_error(error_count)
                        #p_z_error(error_count)
            # 外側
            for i in range(code_distance-1):
                for j in range(code_distance-1):  
                    if i == 0:
                        if j % 2 == 0:
                            CNOT(qubits_d,i,j,qubits_m_out_Z,0,int(j/2))
                            p_x_error(error_count)
                            p_z_error(error_count)
                            p_x_error(error_count)
                            #p_z_error(qubits_m_out_Z,0,int(i/2),p_list[5])
                            CNOT(qubits_d,i,j+1,qubits_m_out_Z,0,int(j/2))
                            p_x_error(error_count)
                            p_z_error(error_count)
                            p_x_error(error_count)
                            #p_z_error(qubits_m_out_Z,0,int(i/2),p_list[5])
                    if i == code_distance-2:
                        if j % 2 == 1:
                            CNOT(qubits_d,code_distance-1,j,qubits_m_out_Z,1,int((j-1)/2))
                            p_x_error(error_count)
                            p_z_error(error_count)
                            p_x_error(error_count)
                            #p_z_error(qubits_m_out_Z,1,int((i-1)/2),p_list[5])
                            CNOT(qubits_d,code_distance-1,j+1,qubits_m_out_Z,1,int((j-1)/2))
                            p_x_error(error_count)
                            p_z_error(error_count)
                            p_x_error(error_count)
                            #p_z_error(qubits_m_out_Z,1,int((i-1)/2),p_list[5]) 
    
        ########################################
        # エラーの履歴
        for i in range(code_distance):
            for j in range(code_distance):
                qubits_d_X[num+1][i][j] =  qubits_d[0][i][j]
                qubits_d_Z[num+1][i][j] =  qubits_d[1][i][j]

        ### 測定結果の格納 & 初期化
        ### Zシンドローム ###
        if num == round_sur - 1:
            # 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j)%2 == 1: ### Xシンドローム
                        #p_x_error(error_count) # 測定前のエラー
                        syndrome_in_Z[1][i][j] =  qubits_m_in[0][i][j] # Xを格納
                        qubits_m_in[0][i][j] = 0
                        qubits_m_in[1][i][j] = 0
            # 外側
            for i in range(int((code_distance-1)/2)):
                #p_x_error(qubits_m_out_Z,0,i,p_list[5]) # 測定前のエラー
                syndrome_out_Z[1][0][i] =  qubits_m_out_Z[0][0][i] # 上
                qubits_m_out_Z[0][0][i] = 0
                qubits_m_out_Z[1][0][i] = 0
                #p_x_error(qubits_m_out_Z,1,i,p_list[5]) # 測定前のエラー
                syndrome_out_Z[1][1][i] =  qubits_m_out_Z[0][1][i] # 下
                qubits_m_out_Z[0][1][i] = 0
                qubits_m_out_Z[1][1][i] = 0
        ### Xシンドローム ###
        # 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j)%2 == 0: ### Xシンドローム
                    H(qubits_m_in,i,j)
                    p_x_error(error_count)
                    p_z_error(error_count)
                    syndrome_in_X[num+1][i][j] =  qubits_m_in[0][i][j] # Zを格納
                    qubits_m_in[0][i][j] = 0
                    qubits_m_in[1][i][j] = 0
        # 外側
        for i in range(int((code_distance-1)/2)):
            H(qubits_m_out_X,0,i)
            p_x_error(error_count) #測定結果反転 (本番では消す)
            p_z_error(error_count) #測定結果反転 (本番では消す)
            syndrome_out_X[num+1][0][i] =  qubits_m_out_X[0][0][i] # 左
            qubits_m_out_X[0][0][i] = 0
            qubits_m_out_X[1][0][i] = 0
            H(qubits_m_out_X,1,i)
            p_x_error(error_count) #測定結果反転 (本番では消す)
            p_z_error(error_count) #測定結果反転 (本番では消す)
            syndrome_out_X[num+1][1][i] =  qubits_m_out_X[0][1][i] # 右
            qubits_m_out_X[0][1][i] = 0
            qubits_m_out_X[1][1][i] = 0

    ###################  ループ終了 #################

    ### 測定結果からシンドロームを計算する ##################

    ### Zシンドローム
    # 内側
    for i in range(code_distance-1):
        for j in range(code_distance-1):
            if (i+j)%2 == 1: 
                syndrome_in_Z[2][i][j] =  (qubits_d[0][i][j]+qubits_d[0][i][j+1]+qubits_d[0][i+1][j]+qubits_d[0][i+1][j+1]) % 2
    # 外側
    for i in range(int((code_distance-1)/2)):
        # 上
        syndrome_out_Z[2][0][i] = (qubits_d[0][0][2*i]+qubits_d[0][0][2*i+1]) % 2
        # 下
        syndrome_out_Z[2][1][i] = (qubits_d[0][code_distance-1][2*i+1]+qubits_d[0][code_distance-1][2*i+2]) % 2
    ### データ量子ビットをresult_dataに移す
    result_data_X = np.zeros((code_distance, code_distance))
    for i in range(code_distance):
        for j in range(code_distance):
            result_data_X[i][j] = qubits_d[0][i][j]
            qubits_d_X[round_sur+1][i][j] =  qubits_d[0][i][j]

    ### Xシンドローム
    for i in range(code_distance):
        for j in range(code_distance):
            H(qubits_d,i,j)
            p_x_error(error_count)
            p_z_error(error_count)
    # 内側
    for i in range(code_distance-1):
        for j in range(code_distance-1):
            if (i+j)%2 == 0: 
                syndrome_in_X[round_sur+1][i][j] =  (qubits_d[0][i][j]+qubits_d[0][i][j+1]+qubits_d[0][i+1][j]+qubits_d[0][i+1][j+1]) % 2
    # 外側
    for i in range(int((code_distance-1)/2)):
        # 右
        syndrome_out_X[round_sur+1][1][i] = (qubits_d[0][2*i][code_distance-1]+qubits_d[0][2*i+1][code_distance-1]) % 2
        # 左
        syndrome_out_X[round_sur+1][0][i] = (qubits_d[0][2*i+1][0]+qubits_d[0][2*i+2][0]) % 2
    ### データ量子ビットをresult_dataに移す
    result_data_Z = np.zeros((code_distance, code_distance))
    for i in range(code_distance):
        for j in range(code_distance):
            result_data_Z[i][j] = qubits_d[0][i][j]
            qubits_d_Z[round_sur+1][i][j] =  qubits_d[0][i][j]
    
    #############  データビットの測定終了  ###############

    ############# detection eventの計算 ###############
    detection_event_in_X = np.zeros((round_sur+1, code_distance-1, code_distance-1))
    detection_event_in_Z = np.zeros((2, code_distance-1, code_distance-1))
    detection_event_out_X = np.zeros((round_sur+1, 2, int((code_distance-1)/2)))
    detection_event_out_Z = np.zeros((2, 2, int((code_distance-1)/2)))

    ### Xシンドローム
    for num in range(round_sur+1):
        ### 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j) % 2 == 0:
                    detection_event_in_X[num,i,j] = (syndrome_in_X[num][i][j] + syndrome_in_X[num+1][i][j]) % 2
        ### 外側
        for i in range(int((code_distance-1)/2)):
            detection_event_out_X[num,0,i] = (syndrome_out_X[num,0,i] + syndrome_out_X[num+1,0,i]) % 2
            detection_event_out_X[num,1,i] = (syndrome_out_X[num,1,i] + syndrome_out_X[num+1,1,i]) % 2
    ### Zシンドローム
    for num in range(2):
        ### 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j) % 2 == 1:
                    detection_event_in_Z[num,i,j] = (syndrome_in_Z[num][i][j] + syndrome_in_Z[num+1][i][j]) % 2
        ### 外側
        for i in range(int((code_distance-1)/2)):
            detection_event_out_Z[num,0,i] = (syndrome_out_Z[num,0,i] + syndrome_out_Z[num+1,0,i]) % 2
            detection_event_out_Z[num,1,i] = (syndrome_out_Z[num,1,i] + syndrome_out_Z[num+1,1,i]) % 2

    ######################################################################## 2週目 ###########################################################################################################

    print(error_count)

    def p_x_error(qubit,i,j,p,error_num):
        if error_num == 0:
            x_error(qubit,i,j)
            error_num -= 1
        elif error_num > 0:
            error_num -= 1

    def p_z_error(qubit,i,j,p,error_num):
        if error_num == 0:
            z_error(qubit,i,j)
            error_num -= 1
        elif error_num > 0:
            error_num -= 1
    
    for error_num in range(error_count):

        qubits_d = np.zeros((2,code_distance,code_distance)) #データ量子ビットの格納
        qubits_d_X = np.zeros((round_sur+2,code_distance,code_distance)) #全体でのXエラーの履歴
        qubits_d_Z = np.zeros((round_sur+2,code_distance,code_distance)) #全体でのXエラーの履歴
        qubits_m_in = np.zeros((2,code_distance-1,code_distance-1)) #測定量子ビット(中)の数
        qubits_m_out_X = np.zeros((2,2,int((code_distance-1)/2))) #測定量子ビット(外)の数
        qubits_m_out_Z = np.zeros((2,2,int((code_distance-1)/2))) #測定量子ビット(外)の数

        syndrome_in_X = np.zeros((round_sur+2, code_distance-1, code_distance-1)) #シンドローム測定の回数+最初の状態のシンドローム+最後の測定から計算したシンドローム
        syndrome_in_Z = np.zeros((3, code_distance-1, code_distance-1)) #最初、最後のシンドローム測定、最後のデータの測定のみ
        syndrome_out_X = np.zeros((round_sur+2,2,int((code_distance-1)/2)))
        syndrome_out_Z = np.zeros((3,2,int((code_distance-1)/2)))

        #############  ループ部分  ##################

        for num in range(round_sur):
            
            ### 反復符号でのエラー
            for i in range(code_distance):
                for j in range(code_distance):
                    p_x_error(qubits_d,i,j,p_list[0],error_num)
                    p_z_error(qubits_d,i,j,p_list[1],error_num)
            
            ### アンシラにエラー(Xシンドロームのみ)
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j) % 2 == 0:
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
            for i in range(int((code_distance-1)/2)):
                p_x_error(qubits_m_out_X,0,i,p_list[5],error_num)
                p_z_error(qubits_m_out_X,0,i,p_list[6],error_num)
                p_x_error(qubits_m_out_X,1,i,p_list[5],error_num)
                p_z_error(qubits_m_out_X,1,i,p_list[6],error_num)

            ### Xシンドローム
            # 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j)%2 == 0: 
                        CNOT(qubits_m_in,i,j,qubits_d,i,j)
                        p_x_error(qubits_d,i,j,p_list[5],error_num)
                        p_z_error(qubits_d,i,j,p_list[6],error_num)
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
                        CNOT(qubits_m_in,i,j,qubits_d,i,j+1,error_num)
                        p_x_error(qubits_d,i,j+1,p_list[6],error_num)
                        p_z_error(qubits_d,i,j+1,p_list[6],error_num)
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
                        CNOT(qubits_m_in,i,j,qubits_d,i+1,j)
                        p_x_error(qubits_d,i+1,j,p_list[5],error_num)
                        p_z_error(qubits_d,i+1,j,p_list[6],error_num)
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
                        CNOT(qubits_m_in,i,j,qubits_d,i+1,j+1)
                        p_x_error(qubits_d,i+1,j+1,p_list[5],error_num)
                        p_z_error(qubits_d,i+1,j+1,p_list[6],error_num)
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
            # 外側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if j == 0:
                        if i % 2 == 1:
                            CNOT(qubits_m_out_X,0,int((i-1)/2),qubits_d,i,j)
                            p_x_error(qubits_d,i,j,p_list[2],error_num)
                            p_z_error(qubits_d,i,j,p_list[3],error_num)
                            p_x_error(qubits_m_out_X,0,int((i-1)/2),p_list[4],error_num)
                            p_z_error(qubits_m_out_X,0,int((i-1)/2),p_list[4],error_num)
                            CNOT(qubits_m_out_X,0,int((i-1)/2),qubits_d,i+1,j)
                            p_x_error(qubits_d,i+1,j,p_list[2],error_num)
                            p_z_error(qubits_d,i+1,j,p_list[3],error_num)
                            p_x_error(qubits_m_out_X,0,int((i-1)/2),p_list[4],error_num)
                            p_z_error(qubits_m_out_X,0,int((i-1)/2),p_list[4],error_num)
                    if j == code_distance-2:
                        if i % 2 == 0:
                            CNOT(qubits_m_out_X,1,int(i/2),qubits_d,i,code_distance-1)
                            p_x_error(qubits_d,i,code_distance-1,p_list[2],error_num)
                            p_z_error(qubits_d,i,code_distance-1,p_list[3],error_num)
                            p_x_error(qubits_m_out_X,1,int(i/2),p_list[4],error_num)
                            p_z_error(qubits_m_out_X,1,int(i/2),p_list[4],error_num)
                            CNOT(qubits_m_out_X,1,int(i/2),qubits_d,i+1,code_distance-1)
                            p_x_error(qubits_d,i+1,code_distance-1,p_list[2],error_num)
                            p_z_error(qubits_d,i+1,code_distance-1,p_list[3],error_num)
                            p_x_error(qubits_m_out_X,1,int(i/2),p_list[4],error_num)  
                            p_z_error(qubits_m_out_X,1,int(i/2),p_list[4],error_num)   

            #Zシンドローム(最後だけ)
            if num == round_sur - 1:
                # 内側
                for i in range(code_distance-1):
                    for j in range(code_distance-1):    
                        if (i+j)%2 == 1: 
                            CNOT(qubits_d,i,j,qubits_m_in,i,j)
                            p_x_error(qubits_d,i,j,p_list[2],error_num)
                            p_z_error(qubits_d,i,j,p_list[3],error_num)
                            p_x_error(qubits_m_in,i,j,p_list[4],error_num)
                            #p_z_error(qubits_m_in,i,j,p_list[3])
                            CNOT(qubits_d,i,j+1,qubits_m_in,i,j)
                            p_x_error(qubits_d,i,j+1,p_list[2],error_num)
                            p_z_error(qubits_d,i,j+1,p_list[3],error_num)
                            p_x_error(qubits_m_in,i,j,p_list[4],error_num)
                            #p_z_error(qubits_m_in,i,j,p_list[3])
                            CNOT(qubits_d,i+1,j,qubits_m_in,i,j)
                            p_x_error(qubits_d,i+1,j,p_list[2],error_num)
                            p_z_error(qubits_d,i+1,j,p_list[3],error_num)
                            p_x_error(qubits_m_in,i,j,p_list[4],error_num)
                            #p_z_error(qubits_m_in,i,j,p_list[3])
                            CNOT(qubits_d,i+1,j+1,qubits_m_in,i,j)
                            p_x_error(qubits_d,i+1,j+1,p_list[2],error_num)
                            p_z_error(qubits_d,i+1,j+1,p_list[3],error_num)
                            p_x_error(qubits_m_in,i,j,p_list[4],error_num)
                            #p_z_error(qubits_m_in,i,j,p_list[3])
                # 外側
                for i in range(code_distance-1):
                    for j in range(code_distance-1):  
                        if i == 0:
                            if j % 2 == 0:
                                CNOT(qubits_d,i,j,qubits_m_out_Z,0,int(j/2))
                                p_x_error(qubits_d,i,j,p_list[2],error_num)
                                p_z_error(qubits_d,i,j,p_list[5],error_num)
                                p_x_error(qubits_m_out_Z,0,int(i/2),p_list[4],error_num)
                                #p_z_error(qubits_m_out_Z,0,int(i/2),p_list[5])
                                CNOT(qubits_d,i,j+1,qubits_m_out_Z,0,int(j/2))
                                p_x_error(qubits_d,i,j+1,p_list[2],error_num)
                                p_z_error(qubits_d,i,j+1,p_list[5],error_num)
                                p_x_error(qubits_m_out_Z,0,int(i/2),p_list[4],error_num)
                                #p_z_error(qubits_m_out_Z,0,int(i/2),p_list[5])
                        if i == code_distance-2:
                            if j % 2 == 1:
                                CNOT(qubits_d,code_distance-1,j,qubits_m_out_Z,1,int((j-1)/2),error_num)
                                p_x_error(qubits_d,code_distance-1,code_distance-1,p_list[2],error_num)
                                p_z_error(qubits_d,code_distance-1,code_distance-1,p_list[3],error_num)
                                p_x_error(qubits_m_out_Z,1,int((i-1)/2),p_list[4],error_num)
                                #p_z_error(qubits_m_out_Z,1,int((i-1)/2),p_list[5])
                                CNOT(qubits_d,code_distance-1,j+1,qubits_m_out_Z,1,int((j-1)/2),error_num)
                                p_x_error(qubits_d,code_distance-1,code_distance-1,p_list[2],error_num)
                                p_z_error(qubits_d,code_distance-1,code_distance-1,p_list[3],error_num)
                                p_x_error(qubits_m_out_Z,1,int((i-1)/2),p_list[4],error_num)
                                #p_z_error(qubits_m_out_Z,1,int((i-1)/2),p_list[5]) 
        
            ########################################
            # エラーの履歴
            for i in range(code_distance):
                for j in range(code_distance):
                    qubits_d_X[num+1][i][j] =  qubits_d[0][i][j]
                    qubits_d_Z[num+1][i][j] =  qubits_d[1][i][j]

            ### 測定結果の格納 & 初期化
            ### Zシンドローム ###
            if num == round_sur - 1:
                # 内側
                for i in range(code_distance-1):
                    for j in range(code_distance-1):
                        if (i+j)%2 == 1: ### Xシンドローム
                            #p_x_error(qubits_m_in,i,j,p_list[5]) # 測定前のエラー
                            syndrome_in_Z[1][i][j] =  qubits_m_in[0][i][j] # Xを格納
                            qubits_m_in[0][i][j] = 0
                            qubits_m_in[1][i][j] = 0
                # 外側
                for i in range(int((code_distance-1)/2)):
                    #p_x_error(qubits_m_out_Z,0,i,p_list[5]) # 測定前のエラー
                    syndrome_out_Z[1][0][i] =  qubits_m_out_Z[0][0][i] # 上
                    qubits_m_out_Z[0][0][i] = 0
                    qubits_m_out_Z[1][0][i] = 0
                    #p_x_error(qubits_m_out_Z,1,i,p_list[5]) # 測定前のエラー
                    syndrome_out_Z[1][1][i] =  qubits_m_out_Z[0][1][i] # 下
                    qubits_m_out_Z[0][1][i] = 0
                    qubits_m_out_Z[1][1][i] = 0
            ### Xシンドローム ###
            # 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j)%2 == 0: ### Xシンドローム
                        H(qubits_m_in,i,j)
                        p_x_error(qubits_m_in,i,j,p_list[5],error_num)
                        p_z_error(qubits_m_in,i,j,p_list[6],error_num)
                        syndrome_in_X[num+1][i][j] =  qubits_m_in[0][i][j] # Zを格納
                        qubits_m_in[0][i][j] = 0
                        qubits_m_in[1][i][j] = 0
            # 外側
            for i in range(int((code_distance-1)/2)):
                H(qubits_m_out_X,0,i)
                p_x_error(qubits_m_out_X,0,i,p_list[5],error_num) #測定結果反転 (本番では消す)
                p_z_error(qubits_m_out_X,0,i,p_list[6],error_num) #測定結果反転 (本番では消す)
                syndrome_out_X[num+1][0][i] =  qubits_m_out_X[0][0][i] # 左
                qubits_m_out_X[0][0][i] = 0
                qubits_m_out_X[1][0][i] = 0
                H(qubits_m_out_X,1,i)
                p_x_error(qubits_m_out_X,1,i,p_list[5],error_num) #測定結果反転 (本番では消す)
                p_z_error(qubits_m_out_X,1,i,p_list[6],error_num) #測定結果反転 (本番では消す)
                syndrome_out_X[num+1][1][i] =  qubits_m_out_X[0][1][i] # 右
                qubits_m_out_X[0][1][i] = 0
                qubits_m_out_X[1][1][i] = 0

        ###################  ループ終了 #################

        ### 測定結果からシンドロームを計算する ##################

        ### Zシンドローム
        # 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j)%2 == 1: 
                    syndrome_in_Z[2][i][j] =  (qubits_d[0][i][j]+qubits_d[0][i][j+1]+qubits_d[0][i+1][j]+qubits_d[0][i+1][j+1]) % 2
        # 外側
        for i in range(int((code_distance-1)/2)):
            # 上
            syndrome_out_Z[2][0][i] = (qubits_d[0][0][2*i]+qubits_d[0][0][2*i+1]) % 2
            # 下
            syndrome_out_Z[2][1][i] = (qubits_d[0][code_distance-1][2*i+1]+qubits_d[0][code_distance-1][2*i+2]) % 2
        ### データ量子ビットをresult_dataに移す
        result_data_X = np.zeros((code_distance, code_distance))
        for i in range(code_distance):
            for j in range(code_distance):
                result_data_X[i][j] = qubits_d[0][i][j]
                qubits_d_X[round_sur+1][i][j] =  qubits_d[0][i][j]

        ### Xシンドローム
        for i in range(code_distance):
            for j in range(code_distance):
                H(qubits_d,i,j)
                p_x_error(qubits_d,i,j,p_list[5],error_num)
                p_z_error(qubits_d,i,j,p_list[6],error_num)
        # 内側
        for i in range(code_distance-1):
            for j in range(code_distance-1):
                if (i+j)%2 == 0: 
                    syndrome_in_X[round_sur+1][i][j] =  (qubits_d[0][i][j]+qubits_d[0][i][j+1]+qubits_d[0][i+1][j]+qubits_d[0][i+1][j+1]) % 2
        # 外側
        for i in range(int((code_distance-1)/2)):
            # 右
            syndrome_out_X[round_sur+1][1][i] = (qubits_d[0][2*i][code_distance-1]+qubits_d[0][2*i+1][code_distance-1]) % 2
            # 左
            syndrome_out_X[round_sur+1][0][i] = (qubits_d[0][2*i+1][0]+qubits_d[0][2*i+2][0]) % 2
        ### データ量子ビットをresult_dataに移す
        result_data_Z = np.zeros((code_distance, code_distance))
        for i in range(code_distance):
            for j in range(code_distance):
                result_data_Z[i][j] = qubits_d[0][i][j]
                qubits_d_Z[round_sur+1][i][j] =  qubits_d[0][i][j]
        
        #############  データビットの測定終了  ###############

        ############# detection eventの計算 ###############
        detection_event_in_X = np.zeros((round_sur+1, code_distance-1, code_distance-1))
        detection_event_in_Z = np.zeros((2, code_distance-1, code_distance-1))
        detection_event_out_X = np.zeros((round_sur+1, 2, int((code_distance-1)/2)))
        detection_event_out_Z = np.zeros((2, 2, int((code_distance-1)/2)))

        ### Xシンドローム
        for num in range(round_sur+1):
            ### 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j) % 2 == 0:
                        detection_event_in_X[num,i,j] = (syndrome_in_X[num][i][j] + syndrome_in_X[num+1][i][j]) % 2
            ### 外側
            for i in range(int((code_distance-1)/2)):
                detection_event_out_X[num,0,i] = (syndrome_out_X[num,0,i] + syndrome_out_X[num+1,0,i]) % 2
                detection_event_out_X[num,1,i] = (syndrome_out_X[num,1,i] + syndrome_out_X[num+1,1,i]) % 2
        ### Zシンドローム
        for num in range(2):
            ### 内側
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if (i+j) % 2 == 1:
                        detection_event_in_Z[num,i,j] = (syndrome_in_Z[num][i][j] + syndrome_in_Z[num+1][i][j]) % 2
            ### 外側
            for i in range(int((code_distance-1)/2)):
                detection_event_out_Z[num,0,i] = (syndrome_out_Z[num,0,i] + syndrome_out_Z[num+1,0,i]) % 2
                detection_event_out_Z[num,1,i] = (syndrome_out_Z[num,1,i] + syndrome_out_Z[num+1,1,i]) % 2

        ############# data qubitでエラーが起こった場所の確認 ##

        dif_qubits_d_X = np.zeros((round_sur+1,code_distance,code_distance))
        dif_qubits_d_Z = np.zeros((round_sur+1,code_distance,code_distance))
        for num in range(round_sur+1):
            for i in range(code_distance):
                for j in range(code_distance):
                    dif_qubits_d_X[num][i][j] = (qubits_d_X[num][i][j] + qubits_d_X[num+1][i][j]) % 2
                dif_qubits_d_Z[num][i][j] = (qubits_d_Z[num][i][j] + qubits_d_Z[num+1][i][j]) % 2

        ########## シンドローム1の点の追加 ############

        edge_of_decoder_graph_X = []

        ### 内側
        for num in range(round_sur+1):
            for i in range(code_distance-1):
                for j in range(code_distance-1):
                    if detection_event_in_X[num,i,j] == 1 and (i+j)%2==0 :
                        edge_of_decoder_graph_X.append((num,i,j)) 
        ### 外側
        for num in range(round_sur+1):
            for i in range(int((code_distance-1)/2)):
                if detection_event_out_X[num,1,i] == 1:
                    edge_of_decoder_graph_X.append((num,2*i,code_distance-1))
                if detection_event_out_X[num,0,i] == 1:
                    edge_of_decoder_graph_X.append((num,2*i+1,-1))

        print(edge_of_decoder_graph_X)
        print(error_num)
    
    #return # 最後の重みづきグラフ

In [29]:
error_count = 0
generate_weight(code_distance=5,p_list=[1,1,1,1,1,1,1],round_sur=1)

UnboundLocalError: local variable 'p_x_error' referenced before assignment